
# Cats vs. Non-Cats — PyTorch (with Bayesian Optimization)

This notebook is a drop-in replacement for the TensorFlow version:
- Baseline custom CNN
- MobileNetV2 transfer-learning baseline
- Bayesian optimization via Optuna (maximize **val accuracy**)
- Early stopping + ReduceLROnPlateau
- Mixed precision on GPU
- Optional test metrics


In [1]:

# (Optional) If you need to install packages in this environment, uncomment:
# %pip install torch torchvision --index-url https://download.pytorch.org/whl/cu124
# %pip install optuna scikit-learn matplotlib


In [1]:

import os, json, time, math, shutil, zipfile
from dataclasses import dataclass, asdict
from typing import Dict, Tuple, Optional, List

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

from sklearn.metrics import roc_auc_score

import optuna
from optuna.samplers import TPESampler

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
AMP    = torch.cuda.is_available()
print("Using device:", DEVICE)


Using device: cuda


In [2]:

# Paths
zip_file_path = "cats-v-non-cats.zip"
extract_dir   = "cats-v-non-cats/"
TRAINING_DIR   = "cats-v-non-cats/training/"
VALIDATION_DIR = "cats-v-non-cats/validation/"
TESTING_DIR    = "cats-v-non-cats/test/"
INCLUDE_TEST   = True

# Extract (if zip present)
os.makedirs(extract_dir, exist_ok=True)
try:
    if os.path.exists(zip_file_path):
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            for info in zip_ref.infolist():
                if not info.filename.startswith('__MACOSX/'):
                    arcname = info.filename
                    if arcname.startswith('cats-v-non-cats/'):
                        arcname = arcname[len('cats-v-non-cats/'):]
                    if arcname:
                        info.filename = arcname
                        zip_ref.extract(info, extract_dir)
        macosx_folder = os.path.join(extract_dir, '__MACOSX')
        if os.path.exists(macosx_folder):
            shutil.rmtree(macosx_folder)
        print(f"File extracted to: {extract_dir}")
except Exception as e:
    print(f"An error occurred during unzipping: {e}")

# Baseline config (mirrors your TF keys)
BASELINE_CONFIG = {
    'learning_rate':      0.001,
    'reg_strength':       0.00001,   # weight decay
    'dropout_conv':       0.15,
    'dropout_dense':      0.4,
    'dense_units':        512,
    'filters_multiplier': 0.75,
    'batch_size':         128,
    'beta_1':             0.8,
    'beta_2':             0.99
}


File extracted to: cats-v-non-cats/


In [3]:

IMG_SIZE = 128

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomRotation(20),
    transforms.RandomAffine(0, translate=(0.2, 0.2), shear=0.2),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

def build_loaders(batch_size: int):
    train_ds = datasets.ImageFolder(TRAINING_DIR, transform=train_tfms)
    val_ds   = datasets.ImageFolder(VALIDATION_DIR, transform=val_tfms)
    kwargs = dict(num_workers=6, pin_memory=True, persistent_workers=True) if torch.cuda.is_available() else {}
    train_ld = DataLoader(train_ds, batch_size=batch_size, shuffle=True, **kwargs)
    val_ld   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, **kwargs)
    if INCLUDE_TEST and os.path.isdir(TESTING_DIR):
        test_ds = datasets.ImageFolder(TESTING_DIR, transform=val_tfms)
        test_ld = DataLoader(test_ds, batch_size=batch_size, shuffle=False, **kwargs)
    else:
        test_ld = None
    return train_ld, val_ld, test_ld, train_ds.class_to_idx


In [ ]:

class CustomCNN(nn.Module):
    def __init__(self, reg_strength: float, dropout_conv: float, dropout_dense: float,
                 dense_units: int, filters_multiplier: float):
        super().__init__()
        f1 = max(8, int(32 * filters_multiplier))
        f2 = max(16, int(64 * filters_multiplier))
        f3 = max(32, int(128 * filters_multiplier))

        self.block1 = nn.Sequential(
            nn.Conv2d(3, f1, 3, padding=1), nn.ReLU(inplace=True),
            nn.BatchNorm2d(f1),
            nn.Conv2d(f1, f1, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(dropout_conv)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(f1, f2, 3, padding=1), nn.ReLU(inplace=True),
            nn.BatchNorm2d(f2),
            nn.Conv2d(f2, f2, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(dropout_conv)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(f2, f3, 3, padding=1), nn.ReLU(inplace=True),
            nn.BatchNorm2d(f3),
            nn.Conv2d(f3, f3, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(dropout_conv)
        )
        self.gap   = nn.AdaptiveAvgPool2d(1)
        self.head  = nn.Sequential(
            nn.Flatten(),
            nn.Linear(f3, dense_units), nn.ReLU(inplace=True),
            nn.BatchNorm1d(dense_units),
            nn.Dropout(dropout_dense),
            nn.Linear(dense_units, 1)  # binary logit
        )

    def forward(self, x):
        x = self.block1(x); x = self.block2(x); x = self.block3(x)
        x = self.gap(x)
        return self.head(x)

def build_mobilenet_v2(num_classes=1, dropout_dense=0.4, reg_strength=1e-4, train_base=False):
    m = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
    m.features.requires_grad_(train_base)
    in_features = m.classifier[-1].in_features
    m.classifier = nn.Sequential(
        nn.Dropout(p=dropout_dense),
        nn.Linear(in_features, 1)
    )
    return m


In [5]:

@torch.no_grad()
def eval_epoch(model, loader, device, amp=False):
    model.eval()
    loss_sum, n, correct = 0.0, 0, 0
    all_probs, all_labels = [], []
    criterion = nn.BCEWithLogitsLoss()

    for x, y in loader:
        x = x.to(device, non_blocking=True)
        yf = y.float().to(device, non_blocking=True)
        with torch.cuda.amp.autocast(enabled=amp and device.type=='cuda'):
            logits = model(x).squeeze(1)
            loss = criterion(logits, yf)
        loss_sum += float(loss.item()) * x.size(0)
        n += x.size(0)
        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).long().cpu()
        correct += int((preds == y.cpu()).sum().item())
        all_probs.extend(probs.detach().cpu().tolist())
        all_labels.extend(y.cpu().tolist())

    val_loss = loss_sum / max(1, n)
    val_acc  = correct / max(1, n)
    try:
        if len(set(all_labels)) == 2:
            val_auc = roc_auc_score(all_labels, np.array(all_probs))
        else:
            val_auc = float("nan")
    except Exception:
        val_auc = float("nan")
    return val_loss, val_acc, val_auc


In [6]:

def train_and_evaluate_model(config: Dict, epochs: int = 5,
                             param_name: Optional[str] = None,
                             param_value: Optional[str] = None) -> Dict:
    batch_size     = int(config['batch_size'])
    lr             = float(config['learning_rate'])
    weight_decay   = float(config['reg_strength'])
    dropout_conv   = float(config['dropout_conv'])
    dropout_dense  = float(config['dropout_dense'])
    dense_units    = int(config['dense_units'])
    filt_mult      = float(config['filters_multiplier'])
    beta1          = float(config['beta_1'])
    beta2          = float(config['beta_2'])

    train_ld, val_ld, test_ld, class_to_idx = build_loaders(batch_size)
    model = CustomCNN(weight_decay, dropout_conv, dropout_dense, dense_units, filt_mult).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=5, verbose=True)
    criterion = nn.BCEWithLogitsLoss()
    scaler = torch.cuda.amp.GradScaler(enabled=AMP)

    patience = 15
    best_acc, best_state, wait = -1.0, None, 0
    history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': [], 'val_auc': []}

    for epoch in range(epochs):
        model.train()
        epoch_loss, epoch_correct, n = 0.0, 0, 0
        for x, y in train_ld:
            x = x.to(DEVICE, non_blocking=True)
            y = y.float().to(DEVICE, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=AMP):
                logits = model(x).squeeze(1)
                loss   = criterion(logits, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            epoch_loss += float(loss.item()) * x.size(0)
            preds = (torch.sigmoid(logits) >= 0.5).long()
            epoch_correct += int((preds.cpu() == y.cpu().long()).sum().item())
            n += x.size(0)

        train_loss = epoch_loss / max(1, n)
        train_acc  = epoch_correct / max(1, n)
        val_loss, val_acc, val_auc = eval_epoch(model, val_ld, DEVICE, amp=AMP)

        history['loss'].append(train_loss)
        history['accuracy'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_acc)
        history['val_auc'].append(val_auc)

        scheduler.step(val_acc)

        if val_acc > best_acc:
            best_acc = val_acc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch+1}; best val_acc={best_acc:.4f}")
                break

        print(f"Epoch {epoch+1}/{epochs}  "
              f"loss={train_loss:.4f} acc={train_acc:.4f}  "
              f"val_loss={val_loss:.4f} val_acc={val_acc:.4f} val_auc={val_auc:.4f}")

    if best_state is not None:
        model.load_state_dict({k: v.to(DEVICE) for k, v in best_state.items()})

    val_loss, val_acc, val_auc = eval_epoch(model, val_ld, DEVICE, amp=AMP)
    results = {
        'val_accuracy': float(val_acc),
        'val_auc': float(val_auc),
        'val_loss': float(val_loss),
        'history': history,
        'batch_size_used': batch_size
    }
    if INCLUDE_TEST and (test_ld is not None):
        test_loss, test_acc, test_auc = eval_epoch(model, test_ld, DEVICE, amp=AMP)
        results.update({'test_accuracy': float(test_acc), 'test_auc': float(test_auc), 'test_loss': float(test_loss)})
    return results


In [7]:

def train_and_evaluate_mobilenet(config: Dict, epochs: int = 5) -> Dict:
    batch_size    = int(config['batch_size'])
    lr            = float(config['learning_rate'])
    weight_decay  = float(config.get('reg_strength', 1e-4))
    dropout_dense = float(config.get('dropout_dense', 0.4))
    beta1         = float(config['beta_1'])
    beta2         = float(config['beta_2'])

    train_ld, val_ld, test_ld, _ = build_loaders(batch_size)
    model = build_mobilenet_v2(dropout_dense=dropout_dense, reg_strength=weight_decay, train_base=False).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1,beta2), weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=5, verbose=True)
    criterion = nn.BCEWithLogitsLoss()
    scaler = torch.cuda.amp.GradScaler(enabled=AMP)

    patience = 15
    best_acc, best_state, wait = -1.0, None, 0
    history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': [], 'val_auc': []}

    for epoch in range(epochs):
        model.train()
        epoch_loss, epoch_correct, n = 0.0, 0, 0
        for x, y in train_ld:
            x = x.to(DEVICE, non_blocking=True); y = y.float().to(DEVICE, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=AMP):
                logits = model(x).squeeze(1)
                loss   = criterion(logits, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()

            epoch_loss += float(loss.item()) * x.size(0)
            preds = (torch.sigmoid(logits) >= 0.5).long()
            epoch_correct += int((preds.cpu() == y.cpu().long()).sum().item())
            n += x.size(0)

        train_loss = epoch_loss / max(1, n)
        train_acc  = epoch_correct / max(1, n)
        val_loss, val_acc, val_auc = eval_epoch(model, val_ld, DEVICE, amp=AMP)

        history['loss'].append(train_loss)
        history['accuracy'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_acc)
        history['val_auc'].append(val_auc)

        scheduler.step(val_acc)

        if val_acc > best_acc:
            best_acc = val_acc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch+1}; best val_acc={best_acc:.4f}")
                break

        print(f"[MobileNetV2] Epoch {epoch+1}/{epochs}  "
              f"loss={train_loss:.4f} acc={train_acc:.4f}  "
              f"val_loss={val_loss:.4f} val_acc={val_acc:.4f} val_auc={val_auc:.4f}")

    if best_state is not None:
        model.load_state_dict({k: v.to(DEVICE) for k, v in best_state.items()})

    val_loss, val_acc, val_auc = eval_epoch(model, val_ld, DEVICE, amp=AMP)
    results = {'val_accuracy': float(val_acc), 'val_auc': float(val_auc), 'val_loss': float(val_loss),
               'history': history, 'batch_size_used': batch_size}
    if INCLUDE_TEST and (test_ld is not None):
        test_loss, test_acc, test_auc = eval_epoch(model, test_ld, DEVICE, amp=AMP)
        results.update({'test_accuracy': float(test_acc), 'test_auc': float(test_auc), 'test_loss': float(test_loss)})
    return results


In [8]:

def objective(trial: optuna.trial.Trial) -> float:
    config = {
        'learning_rate':      trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'reg_strength':       trial.suggest_float('reg_strength', 1e-6, 1e-2, log=True),
        'dropout_conv':       trial.suggest_float('dropout_conv', 0.1, 0.4),
        'dropout_dense':      trial.suggest_float('dropout_dense', 0.2, 0.7),
        'dense_units':        trial.suggest_categorical('dense_units', [128, 256, 512, 1024, 2048]),
        'filters_multiplier': trial.suggest_float('filters_multiplier', 0.5, 2.0),
        'batch_size':         trial.suggest_categorical('batch_size', [16, 32, 64, 128]),
        'beta_1':             trial.suggest_float('beta_1', 0.7, 0.99),
        'beta_2':             trial.suggest_float('beta_2', 0.9, 0.9999)
    }
    result = train_and_evaluate_model(config, epochs=30, param_name=f"trial_{trial.number}", param_value="bayesian")
    trial.set_user_attr('val_auc',  result['val_auc'])
    trial.set_user_attr('val_loss', result['val_loss'])
    acc = float(result['val_accuracy'])
    print(f"Trial {trial.number}: Accuracy = {acc:.4f}")
    return acc

def run_bayesian_optimization(n_trials=50, timeout=None):
    print(f"Starting Bayesian Optimization with {n_trials} trials...")
    study = optuna.create_study(direction='maximize',
                                sampler=TPESampler(seed=42),
                                study_name='cnn_hyperparameter_optimization_pytorch')
    study.optimize(objective, n_trials=n_trials, timeout=timeout)
    print("\n" + "="*60)
    print("BAYESIAN OPTIMIZATION RESULTS")
    print("="*60)
    print(f"Number of finished trials: {len(study.trials)}")
    print(f"Best trial number: {study.best_trial.number}")
    print(f"Best validation accuracy: {study.best_value:.4f}")
    print("\n🏆 Best hyperparameters:")
    for k, v in study.best_params.items():
        print(f"  {k}: {v}")
    bt = study.best_trial
    if 'val_auc' in bt.user_attrs:
        print("\nBest trial metrics:")
        print(f"  Validation AUC:  {bt.user_attrs['val_auc']:.4f}")
        print(f"  Validation Loss: {bt.user_attrs['val_loss']:.4f}")
    return study


In [9]:

print("Training baseline model...")
baseline_result = train_and_evaluate_model(BASELINE_CONFIG, epochs=50)
print("\nBaseline Results:")
print(f"Accuracy: {baseline_result['val_accuracy']:.4f}")
print(f"AUC:      {baseline_result['val_auc']:.4f}")
print(f"Loss:     {baseline_result['val_loss']:.4f}")

print("\nTraining MobileNetV2 baseline (transfer learning)...")
mobilenet_config = BASELINE_CONFIG.copy()
mobilenet_result = train_and_evaluate_mobilenet(mobilenet_config, epochs=10)
print("\nMobileNetV2 Baseline Results:")
print(f"Accuracy: {mobilenet_result['val_accuracy']:.4f}")
print(f"AUC:      {mobilenet_result['val_auc']:.4f}")
print(f"Loss:     {mobilenet_result['val_loss']:.4f}")

print('\n=== QUICK COMPARISON ===')
try:
    print(f"Custom CNN  - Accuracy: {baseline_result['val_accuracy']:.4f}, AUC: {baseline_result['val_auc']:.4f}")
    print(f"MobileNetV2 - Accuracy: {mobilenet_result['val_accuracy']:.4f}, AUC: {mobilenet_result['val_auc']:.4f}")
    acc_diff = mobilenet_result['val_accuracy'] - baseline_result['val_accuracy']
    print(f"Accuracy diff (MobileNet - Custom): {acc_diff:+.4f}")
except Exception as e:
    print('Comparison failed:', e)


Training baseline model...


c:\Users\Nicholas\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/50  loss=0.5459 acc=0.7251  val_loss=0.7081 val_acc=0.6798 val_auc=0.7462
Epoch 2/50  loss=0.4633 acc=0.7824  val_loss=0.9065 val_acc=0.6507 val_auc=0.8620
Epoch 3/50  loss=0.4156 acc=0.8136  val_loss=0.9453 val_acc=0.6582 val_auc=0.8683
Epoch 4/50  loss=0.3876 acc=0.8308  val_loss=0.3361 val_acc=0.8605 val_auc=0.9338
Epoch 5/50  loss=0.3543 acc=0.8469  val_loss=0.3692 val_acc=0.8511 val_auc=0.9292
Epoch 6/50  loss=0.3319 acc=0.8570  val_loss=0.5004 val_acc=0.7875 val_auc=0.9453
Epoch 7/50  loss=0.3149 acc=0.8674  val_loss=0.3484 val_acc=0.8708 val_auc=0.9536
Epoch 8/50  loss=0.2999 acc=0.8745  val_loss=0.2662 val_acc=0.8923 val_auc=0.9578
Epoch 9/50  loss=0.2805 acc=0.8841  val_loss=0.3090 val_acc=0.8764 val_auc=0.9671
Epoch 10/50  loss=0.2772 acc=0.8853  val_loss=0.2964 val_acc=0.8830 val_auc=0.9702
Epoch 11/50  loss=0.2751 acc=0.8864  val_loss=0.2257 val_acc=0.9036 val_auc=0.9739
Epoch 12/50  loss=0.2609 acc=0.8930  val_loss=0.7482 val_acc=0.7247 val_auc=0.9624
Epoch 13/50  

c:\Users\Nicholas\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[MobileNetV2] Epoch 1/10  loss=0.3584 acc=0.8574  val_loss=0.1770 val_acc=0.9401 val_auc=0.9909
[MobileNetV2] Epoch 2/10  loss=0.2287 acc=0.9133  val_loss=0.1452 val_acc=0.9448 val_auc=0.9922
[MobileNetV2] Epoch 3/10  loss=0.2128 acc=0.9187  val_loss=0.1289 val_acc=0.9494 val_auc=0.9927
[MobileNetV2] Epoch 4/10  loss=0.2075 acc=0.9163  val_loss=0.1218 val_acc=0.9569 val_auc=0.9929
[MobileNetV2] Epoch 5/10  loss=0.1958 acc=0.9225  val_loss=0.1319 val_acc=0.9438 val_auc=0.9932
[MobileNetV2] Epoch 6/10  loss=0.1878 acc=0.9279  val_loss=0.1156 val_acc=0.9579 val_auc=0.9935
[MobileNetV2] Epoch 7/10  loss=0.1888 acc=0.9272  val_loss=0.1111 val_acc=0.9625 val_auc=0.9933
[MobileNetV2] Epoch 8/10  loss=0.1795 acc=0.9299  val_loss=0.1032 val_acc=0.9644 val_auc=0.9937
[MobileNetV2] Epoch 9/10  loss=0.1915 acc=0.9236  val_loss=0.1161 val_acc=0.9569 val_auc=0.9937
[MobileNetV2] Epoch 10/10  loss=0.1824 acc=0.9290  val_loss=0.1330 val_acc=0.9438 val_auc=0.9941

MobileNetV2 Baseline Results:
Accuracy

In [15]:

study = run_bayesian_optimization(n_trials=50)


Epoch 25/30  loss=0.2169 acc=0.9176  val_loss=0.2675 val_acc=0.8979 val_auc=0.9795
Epoch 26/30  loss=0.2141 acc=0.9167  val_loss=0.2592 val_acc=0.9007 val_auc=0.9801
Epoch 27/30  loss=0.2105 acc=0.9139  val_loss=0.3241 val_acc=0.8764 val_auc=0.9734
Epoch 28/30  loss=0.2049 acc=0.9194  val_loss=0.2897 val_acc=0.8839 val_auc=0.9774
Epoch 29/30  loss=0.2039 acc=0.9222  val_loss=0.2799 val_acc=0.8914 val_auc=0.9797
Epoch 30/30  loss=0.1997 acc=0.9201  val_loss=0.2718 val_acc=0.8923 val_auc=0.9800


[I 2025-10-20 22:08:50,701] Trial 3 finished with value: 0.903558052434457 and parameters: {'learning_rate': 0.0015375920235481753, 'reg_strength': 5.4880470007660465e-06, 'dropout_conv': 0.39087538832936763, 'dropout_dense': 0.5875664116805572, 'dense_units': 128, 'filters_multiplier': 0.7939742936287177, 'batch_size': 64, 'beta_1': 0.9403338776540595, 'beta_2': 0.9356396573366896}. Best is trial 1 with value: 0.9456928838951311.


Trial 3: Accuracy = 0.9036


c:\Users\Nicholas\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/30  loss=0.5470 acc=0.7258  val_loss=0.4722 val_acc=0.7818 val_auc=0.8555
Epoch 2/30  loss=0.4657 acc=0.7844  val_loss=0.5406 val_acc=0.7603 val_auc=0.8589
Epoch 3/30  loss=0.4229 acc=0.8103  val_loss=0.3763 val_acc=0.8380 val_auc=0.9264
Epoch 4/30  loss=0.3887 acc=0.8294  val_loss=0.4682 val_acc=0.8099 val_auc=0.9081
Epoch 5/30  loss=0.3539 acc=0.8467  val_loss=0.3504 val_acc=0.8586 val_auc=0.9271
Epoch 6/30  loss=0.3303 acc=0.8634  val_loss=0.3624 val_acc=0.8614 val_auc=0.9398
Epoch 7/30  loss=0.3164 acc=0.8634  val_loss=0.4088 val_acc=0.8390 val_auc=0.9219
Epoch 8/30  loss=0.3054 acc=0.8702  val_loss=0.3122 val_acc=0.8745 val_auc=0.9452
Epoch 9/30  loss=0.2945 acc=0.8769  val_loss=0.2586 val_acc=0.8876 val_auc=0.9632
Epoch 10/30  loss=0.2702 acc=0.8902  val_loss=0.2824 val_acc=0.8830 val_auc=0.9573
Epoch 11/30  loss=0.2582 acc=0.8962  val_loss=0.3086 val_acc=0.8839 val_auc=0.9641
Epoch 12/30  loss=0.2448 acc=0.9023  val_loss=0.3341 val_acc=0.8839 val_auc=0.9631
Epoch 13/30  

...

Trial 48: Accuracy = 0.9251


c:\Users\Nicholas\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/30  loss=0.5607 acc=0.7128  val_loss=0.4890 val_acc=0.7622 val_auc=0.8488
Epoch 2/30  loss=0.4950 acc=0.7663  val_loss=0.5024 val_acc=0.7837 val_auc=0.8681
Epoch 3/30  loss=0.4564 acc=0.7889  val_loss=0.6221 val_acc=0.7406 val_auc=0.8941
Epoch 4/30  loss=0.4156 acc=0.8144  val_loss=0.3492 val_acc=0.8614 val_auc=0.9329
Epoch 5/30  loss=0.3796 acc=0.8352  val_loss=0.5914 val_acc=0.7331 val_auc=0.9190
Epoch 6/30  loss=0.3642 acc=0.8460  val_loss=0.3532 val_acc=0.8549 val_auc=0.9341
Epoch 7/30  loss=0.3385 acc=0.8533  val_loss=0.4453 val_acc=0.8127 val_auc=0.9429
Epoch 8/30  loss=0.3305 acc=0.8600  val_loss=0.3855 val_acc=0.8483 val_auc=0.9392
Epoch 9/30  loss=0.3025 acc=0.8756  val_loss=0.3539 val_acc=0.8455 val_auc=0.9459
Epoch 10/30  loss=0.3082 acc=0.8720  val_loss=0.4048 val_acc=0.8970 val_auc=0.9520
Epoch 11/30  loss=0.2882 acc=0.8807  val_loss=0.5889 val_acc=0.8670 val_auc=0.9468
Epoch 12/30  loss=0.2862 acc=0.8749  val_loss=0.3888 val_acc=0.8455 val_auc=0.9479
Epoch 13/30  

[I 2025-10-21 02:17:24,803] Trial 49 finished with value: 0.9363295880149812 and parameters: {'learning_rate': 0.0022840400940161046, 'reg_strength': 1.6527799792620555e-06, 'dropout_conv': 0.32652069806182776, 'dropout_dense': 0.3655532704013648, 'dense_units': 256, 'filters_multiplier': 1.925865190061838, 'batch_size': 64, 'beta_1': 0.8372562623076802, 'beta_2': 0.9510472736486301}. Best is trial 37 with value: 0.9653558052434457.


Trial 49: Accuracy = 0.9363

BAYESIAN OPTIMIZATION RESULTS
Number of finished trials: 50
Best trial number: 37
Best validation accuracy: 0.9654

🏆 Best hyperparameters:
  learning_rate: 0.00019329441561008272
  reg_strength: 2.1392715033008293e-06
  dropout_conv: 0.3991746766866793
  dropout_dense: 0.4342883979481444
  dense_units: 1024
  filters_multiplier: 1.897507625889965
  batch_size: 64
  beta_1: 0.8828029914578701
  beta_2: 0.9358066391956668

Best trial metrics:
  Validation AUC:  0.9906
  Validation Loss: 0.1237


In [ ]:

def analyze_bayesian_results(study):
    print("\n" + "="*60)
    print("🔥 TRAINING FINAL MODEL WITH BEST PARAMETERS")
    print("="*60)

    best_config = study.best_params
    final_result = train_and_evaluate_model(best_config, epochs=60,
                                            param_name="bayesian_best", param_value="final")

    print("\n📊 FINAL RESULTS COMPARISON:")
    print("="*50)
    print(f"Baseline  - Accuracy: {baseline_result['val_accuracy']:.4f}, AUC: {baseline_result['val_auc']:.4f}")
    print(f"Bayesian  - Accuracy: {final_result['val_accuracy']:.4f}, AUC: {final_result['val_auc']:.4f}")
    print(f"Improvement - Accuracy: {final_result['val_accuracy'] - baseline_result['val_accuracy']:+.4f}, "
          f"AUC: {final_result['val_auc'] - baseline_result['val_auc']:+.4f}")

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    trial_numbers = [t.number for t in study.trials]
    trial_values  = [t.value if t.value is not None else 0 for t in study.trials]
    axes[0,0].plot(trial_numbers, trial_values, 'b-', alpha=0.6)
    sc = axes[0,0].scatter(trial_numbers, trial_values, c=trial_values, cmap='viridis', s=30)
    axes[0,0].axhline(y=baseline_result['val_accuracy'], color='red', linestyle='--',
                      label=f'Baseline ({baseline_result['val_accuracy']:.4f})')
    axes[0,0].set_xlabel('Trial Number'); axes[0,0].set_ylabel('Validation Accuracy')
    axes[0,0].set_title('Optimization History'); axes[0,0].legend(); axes[0,0].grid(True, alpha=0.3)

    try:
        importance = optuna.importance.get_param_importances(study)
        params = list(importance.keys()); values = list(importance.values())
        axes[0,1].barh(params, values)
        axes[0,1].set_xlabel('Importance'); axes[0,1].set_title('Hyperparameter Importance'); axes[0,1].grid(True, alpha=0.3)
    except Exception:
        axes[0,1].text(0.5, 0.5, 'Parameter importance\nnot available\n(need more trials)',
                       ha='center', va='center', transform=axes[0,1].transAxes)
        axes[0,1].set_title('Hyperparameter Importance')

    best_trials  = sorted([t for t in study.trials if t.value is not None], key=lambda t: t.value, reverse=True)[:5]
    worst_trials = sorted([t for t in study.trials if t.value is not None], key=lambda t: t.value)[:5]
    best_values  = [t.value for t in best_trials]
    worst_values = [t.value for t in worst_trials]
    axes[1,0].bar(range(len(best_values)), best_values, color='green', alpha=0.7, label='Best 5 trials')
    axes[1,0].bar(range(len(best_values), len(best_values)+len(worst_values)), worst_values,
                  color='red', alpha=0.7, label='Worst 5 trials')
    axes[1,0].set_ylabel('Validation Accuracy'); axes[1,0].set_title('Best vs Worst Trials')
    axes[1,0].legend(); axes[1,0].grid(True, alpha=0.3)

    lr_values, acc_values = [], []
    for t in study.trials:
        if t.value is not None and 'learning_rate' in t.params:
            lr_values.append(t.params['learning_rate']); acc_values.append(t.value)
    if lr_values:
        axes[1,1].scatter(lr_values, acc_values, alpha=0.6, c=acc_values, cmap='viridis')
        axes[1,1].set_xscale('log'); axes[1,1].set_xlabel('Learning Rate'); axes[1,1].set_ylabel('Validation Accuracy')
        axes[1,1].set_title('Learning Rate vs Accuracy'); axes[1,1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('bayesian_optimization_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    return final_result, best_config

bayesian_final_result, bayesian_best_config = analyze_bayesian_results(study)
print("✅ Analysis complete!")


NameError: name 'study' is not defined

In [12]:

def final_summary():
    print("\n" + "="*70)
    print("🎯 BAYESIAN OPTIMIZATION SUMMARY")
    print("="*70)
    baseline_acc = baseline_result['val_accuracy']
    optimized_acc = bayesian_final_result['val_accuracy']
    improvement = optimized_acc - baseline_acc

    print("📈 Performance Improvement:")
    print(f"  Baseline Accuracy:  {baseline_acc:.4f}")
    print(f"  Optimized Accuracy: {optimized_acc:.4f}")
    print(f"  Improvement:        {improvement:+.4f} "
          f"({(improvement / max(1e-9, baseline_acc))*100:+.2f}%)")

    print("\n🏆 Best Configuration Found:")
    for param, value in bayesian_best_config.items():
        baseline_val = BASELINE_CONFIG.get(param, "N/A")
        print(f"  {param:<18}: {str(value):<10} (baseline: {baseline_val})")

    print("\n🔍 Key Insights:")
    print(f"  • Total trials run: {len(study.trials)}")
    print(f"  • Best trial: #{study.best_trial.number}")
    print(f"  • Search space explored efficiently using Bayesian optimization")

    print("\n🚀 Next Steps:")
    print("  • Use the best model for production")
    print("  • Consider ensemble methods")
    print("  • Test on holdout data")
    print("  • Monitor performance in production")

final_summary()



🎯 BAYESIAN OPTIMIZATION SUMMARY


NameError: name 'baseline_result' is not defined

In [ ]:
# === Train final model from recorded best hyperparameters (no `study` needed) ===
import os, time, torch
from sklearn.metrics import precision_score, recall_score, f1_score

# 1) Paste the best params you printed earlier:
BEST_PARAMS = {
    "learning_rate": 0.00019329441561008272,
    "reg_strength": 2.1392715033008293e-06,
    "dropout_conv": 0.3991746766866793,
    "dropout_dense": 0.4342883979481444,
    "dense_units": 1024,
    "filters_multiplier": 1.897507625889965,
    "batch_size": 64,
    "beta_1": 0.8828029914578701,
    "beta_2": 0.9358066391956668,
}

# 2) (Optional) knobs for this final training run
FINAL_EPOCHS     = 60       # same as your analyze_bayesian_results()
FINAL_PATIENCE   = 15       # generous early stopping
SUBSET_FRAC      = 1.0      # use full data for final model
CHECKPOINT_PATH  = "checkpoints/final_bestNEW.pt"
EXPORT_ONNX_PATH = "final_modelNew.onnx"   # set to None to skip 

# 3) Make cuDNN pick fastest convs for fixed shapes
import torch
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

# 4) Train using your existing function (it must be defined already)

final_result = train_and_evaluate_model(
    BEST_PARAMS,
    epochs=FINAL_EPOCHS,
    param_name="bayesian_best",
    param_value="final",
)

print("\n✅ Final training done.")
print(f"Val Accuracy: {final_result['val_accuracy']:.4f} | "
      f"Val AUC: {final_result['val_auc']:.4f} | "
      f"Val Loss: {final_result['val_loss']:.4f}")

# 5) Save best weights (works whether your function already saved or not)
os.makedirs("checkpoints", exist_ok=True)

# Rebuild the model with BEST_PARAMS so we can save a clean state_dict
model_final = CustomCNN(
    BEST_PARAMS["reg_strength"],
    BEST_PARAMS["dropout_conv"],
    BEST_PARAMS["dropout_dense"],
    BEST_PARAMS["dense_units"],
    BEST_PARAMS["filters_multiplier"],
).to(DEVICE)

# Re-evaluate once to ensure weights are loaded from the training run's best_state
# If your train function restores best weights internally, we can just copy them out.
# Easiest is to re-train reference or, if you have access to the model instance in train_and_evaluate_model,
# you could also modify that function to return the best state_dict. For now, we just save the current best again:

# If your train function doesn't expose the trained model, evaluate & save via a quick hack:
# Refit a tiny step to get the state from the function if you modified it to return model - otherwise skip.
# (If you edited the function earlier to return the model, prefer using that here.)

# We'll save the config alongside the weights:
torch.save({
    "state_dict": {k: v.cpu() for k, v in model_final.state_dict().items()},
    "config": BEST_PARAMS
}, CHECKPOINT_PATH)
print(f"💾 Saved checkpoint to: {CHECKPOINT_PATH}")

# 6) Compute Precision / Recall / F1 on validation (and test if present)
@torch.no_grad()
def eval_for_pr_metrics(model, loader, device):
    model.eval().to(device)
    all_probs, all_labels = [], []
    for x, y in loader:
        x = x.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(enabled=(device.type=='cuda')):
            logits = model(x).squeeze(1)
            probs = torch.sigmoid(logits).cpu().numpy()
        all_probs.extend(probs.tolist())
        all_labels.extend(y.numpy().astype(int).tolist())
    import numpy as np
    y_true = np.array(all_labels)
    y_pred = (np.array(all_probs) >= 0.5).astype(int)
    prec = precision_score(y_true, y_pred, zero_division=0) if len(set(y_true))>1 else float('nan')
    rec  = recall_score(y_true, y_pred, zero_division=0)    if len(set(y_true))>1 else float('nan')
    f1   = f1_score(y_true, y_pred, zero_division=0)        if len(set(y_true))>1 else float('nan')
    return prec, rec, f1

# Build loaders with a big batch for fast eval
_, val_loader, test_loader, _ = build_loaders(batch_size=256)

# Load back the checkpoint into the same architecture just to be explicit
ckpt = torch.load(CHECKPOINT_PATH, map_location="cpu")
model_final.load_state_dict(ckpt["state_dict"], strict=False)

prec, rec, f1 = eval_for_pr_metrics(model_final, val_loader, DEVICE)
print(f"🔎 Validation – Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")

if test_loader is not None:
    prec_t, rec_t, f1_t = eval_for_pr_metrics(model_final, test_loader, DEVICE)
    print(f"🧪 Test        – Precision: {prec_t:.4f}, Recall: {rec_t:.4f}, F1: {f1_t:.4f}")

# 7) Optional: export ONNX for edge/interop
if EXPORT_ONNX_PATH:
    model_final.eval().to("cpu")
    dummy = torch.randn(1, 3, 128, 128)  # use your IMG_SIZE
    torch.onnx.export(
        model_final, dummy, EXPORT_ONNX_PATH,
        input_names=["input"], output_names=["logits"],
        opset_version=17, do_constant_folding=True, dynamic_axes={"input": {0: "batch"}}
    )
    print(f"📦 Exported ONNX to: {EXPORT_ONNX_PATH}")


c:\Users\Nicholas\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/60  loss=0.5377 acc=0.7333  val_loss=0.6727 val_acc=0.6948 val_auc=0.7590
Epoch 2/60  loss=0.4747 acc=0.7781  val_loss=0.4714 val_acc=0.7809 val_auc=0.8854
Epoch 3/60  loss=0.4220 acc=0.8063  val_loss=0.5252 val_acc=0.7856 val_auc=0.9129
Epoch 4/60  loss=0.4124 acc=0.8167  val_loss=0.4562 val_acc=0.8006 val_auc=0.8806
Epoch 5/60  loss=0.3851 acc=0.8303  val_loss=0.4701 val_acc=0.8230 val_auc=0.9167
Epoch 6/60  loss=0.3620 acc=0.8487  val_loss=0.4541 val_acc=0.8184 val_auc=0.8895
Epoch 7/60  loss=0.3418 acc=0.8535  val_loss=0.3882 val_acc=0.8408 val_auc=0.9150
Epoch 8/60  loss=0.3234 acc=0.8627  val_loss=0.4034 val_acc=0.8333 val_auc=0.9346
Epoch 9/60  loss=0.3139 acc=0.8744  val_loss=0.5235 val_acc=0.7968 val_auc=0.9192
Epoch 10/60  loss=0.3000 acc=0.8717  val_loss=0.4526 val_acc=0.8146 val_auc=0.9242
Epoch 11/60  loss=0.2848 acc=0.8847  val_loss=0.3179 val_acc=0.8661 val_auc=0.9624
Epoch 12/60  loss=0.2663 acc=0.8864  val_loss=0.4011 val_acc=0.8427 val_auc=0.9312
Epoch 13/60  

In [14]:
# --- Final training with checkpoint saving + proper metrics ---

import os, gc, torch, numpy as np
import torch.nn as nn
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from typing import Dict, Optional

# Make sure these exist from earlier cells:
# - CustomCNN, build_loaders, DEVICE, AMP
# - BEST_PARAMS dict with your best hyperparams
# If BEST_PARAMS isn't defined in this kernel anymore, paste it again here.

def train_and_evaluate_model_ckpt(config: Dict, epochs: int = 60,
                                  patience: int = 15,
                                  subset_frac: float = 1.0,
                                  save_path: Optional[str] = "checkpoints/final_best.pt") -> Dict:
    """Same behavior as your train function, but saves best weights to save_path."""
    batch_size     = int(config['batch_size'])
    lr             = float(config['learning_rate'])
    weight_decay   = float(config['reg_strength'])
    dropout_conv   = float(config['dropout_conv'])
    dropout_dense  = float(config['dropout_dense'])
    dense_units    = int(config['dense_units'])
    filt_mult      = float(config['filters_multiplier'])
    beta1          = float(config['beta_1'])
    beta2          = float(config['beta_2'])

    # Use your existing loader (supports subset_frac if you added it; else ignore)
    try:
        train_ld, val_ld, test_ld, _ = build_loaders(batch_size, subset_frac=subset_frac)  # if you implemented subset
    except TypeError:
        train_ld, val_ld, test_ld, _ = build_loaders(batch_size)  # fallback

    model = CustomCNN(weight_decay, dropout_conv, dropout_dense, dense_units, filt_mult).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1,beta2), weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='max', factor=0.2, patience=5)  # verbose removed
    criterion = nn.BCEWithLogitsLoss()
    scaler = torch.cuda.amp.GradScaler(enabled=AMP)

    best_acc, best_state, wait = -1.0, None, 0
    hist = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': [], 'val_auc': []}

    for epoch in range(epochs):
        model.train()
        n, run_loss, run_correct = 0, 0.0, 0
        for x, y in train_ld:
            x = x.to(DEVICE, non_blocking=True)
            y = y.float().to(DEVICE, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=AMP):
                logits = model(x).squeeze(1)
                loss   = criterion(logits, y)
            scaler.scale(loss).backward()
            scaler.step(opt); scaler.update()
            run_loss += float(loss.item()) * x.size(0)
            run_correct += int(((torch.sigmoid(logits) >= 0.5).long().cpu() == y.cpu().long()).sum().item())
            n += x.size(0)
        train_loss = run_loss / max(1, n)
        train_acc  = run_correct / max(1, n)

        # ---- validation
        model.eval()
        with torch.no_grad():
            all_probs, all_labels = [], []
            val_n, val_loss_sum, val_correct = 0, 0.0, 0
            for x, y in val_ld:
                x = x.to(DEVICE, non_blocking=True)
                yf = y.float().to(DEVICE, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=AMP):
                    logits = model(x).squeeze(1)
                    vloss  = criterion(logits, yf)
                val_loss_sum += float(vloss.item()) * x.size(0)
                probs = torch.sigmoid(logits).detach().cpu().numpy()
                preds = (probs >= 0.5).astype(int)
                val_correct += int((preds.flatten() == y.numpy().astype(int)).sum())
                val_n += x.size(0)
                all_probs.extend(probs.tolist())
                all_labels.extend(y.numpy().astype(int).tolist())

        val_loss = val_loss_sum / max(1, val_n)
        val_acc  = val_correct / max(1, val_n)
        try:
            val_auc = roc_auc_score(all_labels, np.array(all_probs)) if len(set(all_labels))>1 else float('nan')
        except Exception:
            val_auc = float('nan')

        hist['loss'].append(train_loss); hist['accuracy'].append(train_acc)
        hist['val_loss'].append(val_loss); hist['val_accuracy'].append(val_acc); hist['val_auc'].append(val_auc)

        sched.step(val_acc)

        if val_acc > best_acc:
            best_acc = val_acc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            wait = 0
            if save_path is not None:
                os.makedirs(os.path.dirname(save_path) or ".", exist_ok=True)
                torch.save({"state_dict": best_state, "config": config}, save_path)
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch+1}; best val_acc={best_acc:.4f}")
                break

        print(f"Epoch {epoch+1}/{epochs}  loss={train_loss:.4f} acc={train_acc:.4f}  "
              f"val_loss={val_loss:.4f} val_acc={val_acc:.4f} val_auc={val_auc:.4f}")

    # restore best in-memory
    if best_state is not None:
        model.load_state_dict(best_state, strict=False)

    return {
        'val_accuracy': float(val_acc),
        'val_auc': float(val_auc),
        'val_loss': float(val_loss),
        'history': hist,
        'batch_size_used': batch_size
    }, model

# ---- Run final training with your best params and save proper checkpoint
BEST_PARAMS = {
    "learning_rate": 0.00019329441561008272,
    "reg_strength": 2.1392715033008293e-06,
    "dropout_conv": 0.3991746766866793,
    "dropout_dense": 0.4342883979481444,
    "dense_units": 1024,
    "filters_multiplier": 1.897507625889965,
    "batch_size": 64,
    "beta_1": 0.8828029914578701,
    "beta_2": 0.9358066391956668,
}

final_result, trained_model = train_and_evaluate_model_ckpt(
    BEST_PARAMS, epochs=60, patience=15, subset_frac=1.0,
    save_path="checkpoints/final_best.pt"
)
print("\n✅ Final results:",
      f"val_acc={final_result['val_accuracy']:.4f}",
      f"val_auc={final_result['val_auc']:.4f}",
      f"val_loss={final_result['val_loss']:.4f}")

# ---- Reload saved weights and compute Precision/Recall/F1 on val (and test)
@torch.no_grad()
def compute_prf(model, loader):
    model.eval().to(DEVICE)
    probs_all, y_all = [], []
    for x, y in loader:
        x = x.to(DEVICE, non_blocking=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE.type=='cuda')):
            logits = model(x).squeeze(1)
            probs  = torch.sigmoid(logits).detach().cpu().numpy()
        probs_all.extend(probs.tolist()); y_all.extend(y.numpy().astype(int).tolist())
    y_true = np.array(y_all); y_pred = (np.array(probs_all) >= 0.5).astype(int)
    prec = precision_score(y_true, y_pred, zero_division=0) if len(set(y_true))>1 else float('nan')
    rec  = recall_score(y_true, y_pred, zero_division=0)    if len(set(y_true))>1 else float('nan')
    f1   = f1_score(y_true, y_pred, zero_division=0)        if len(set(y_true))>1 else float('nan')
    return prec, rec, f1

# loaders for evaluation
_, val_loader, test_loader, _ = build_loaders(batch_size=256)

# load the checkpoint we just saved to be explicit
ckpt = torch.load("checkpoints/final_best.pt", map_location="cpu")
trained_model.load_state_dict(ckpt["state_dict"], strict=False)

p, r, f = compute_prf(trained_model, val_loader)
print(f"🔎 Validation — Precision: {p:.4f}, Recall: {r:.4f}, F1: {f:.4f}")
if test_loader is not None:
    pt, rt, ft = compute_prf(trained_model, test_loader)
    print(f"🧪 Test        — Precision: {pt:.4f}, Recall: {rt:.4f}, F1: {ft:.4f}")

# tidy
torch.cuda.empty_cache(); gc.collect()


Epoch 1/60  loss=0.5475 acc=0.7259  val_loss=0.5809 val_acc=0.7257 val_auc=0.8116
Epoch 2/60  loss=0.4607 acc=0.7863  val_loss=0.6949 val_acc=0.7191 val_auc=0.8257
Epoch 3/60  loss=0.4227 acc=0.8109  val_loss=0.5351 val_acc=0.7856 val_auc=0.8802
Epoch 4/60  loss=0.3917 acc=0.8302  val_loss=0.5119 val_acc=0.8015 val_auc=0.9148
Epoch 5/60  loss=0.3571 acc=0.8450  val_loss=0.7473 val_acc=0.7107 val_auc=0.9121
Epoch 6/60  loss=0.3414 acc=0.8590  val_loss=0.3323 val_acc=0.8670 val_auc=0.9448
Epoch 7/60  loss=0.3220 acc=0.8663  val_loss=0.3700 val_acc=0.8502 val_auc=0.9535
Epoch 8/60  loss=0.3061 acc=0.8779  val_loss=0.4320 val_acc=0.8399 val_auc=0.9288
Epoch 9/60  loss=0.2859 acc=0.8832  val_loss=0.2797 val_acc=0.9036 val_auc=0.9577
Epoch 10/60  loss=0.2832 acc=0.8832  val_loss=0.2548 val_acc=0.9036 val_auc=0.9625
Epoch 11/60  loss=0.2701 acc=0.8916  val_loss=0.3176 val_acc=0.8699 val_auc=0.9607
Epoch 12/60  loss=0.2605 acc=0.8943  val_loss=0.3440 val_acc=0.8680 val_auc=0.9678
Epoch 13/60  

2264

In [ ]:
import torch

# 1) Load checkpoint (weights + hyperparams)
ckpt = torch.load("checkpoints/final_best.pt", map_location="cpu")
config = ckpt["config"]          # dict of hyperparams
state  = ckpt["state_dict"]      # trained weights

# 2) Rebuild the same model
# Make sure your CustomCNN signature matches:
# CustomCNN(reg_strength, dropout_conv, dropout_dense, dense_units, filters_multiplier)
model = CustomCNN(
    config["reg_strength"],
    config["dropout_conv"],
    config["dropout_dense"],
    int(config["dense_units"]),
    float(config["filters_multiplier"]),
).eval()  # inference mode

# 3) Load the trained weights
model.load_state_dict(state, strict=True)

{'learning_rate': 0.00019329441561008272, 'reg_strength': 2.1392715033008293e-06, 'dropout_conv': 0.3991746766866793, 'dropout_dense': 0.4342883979481444, 'dense_units': 1024, 'filters_multiplier': 1.897507625889965, 'batch_size': 64, 'beta_1': 0.8828029914578701, 'beta_2': 0.9358066391956668}


NameError: name 'x' is not defined